# Project of ana and leo

In [1]:
import pandas as pd
import statistics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from function import Utils
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from pandas.api.types import is_numeric_dtype
import numpy as np
from IPython.display import display

warnings.filterwarnings('ignore')

## Data loading

In [2]:
df = pd.read_csv("hmw/small3insert.csv")

In [3]:
df

,Case,Event,Activity,Timestamp,Resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_anomaly_type,resource_parameter,trace_change_resource,variant_num
0,case_0,9367,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,9368,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,9369,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,9370,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,9371,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358
...,...,...,...,...,...,...,...,...,...,...,...,...
45842,case_999,31364,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_171
45843,case_999,31365,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,0.007068,0,6,normal,NaN,0,var_171
45844,case_999,31366,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,0.018454,0,7,normal,NaN,0,var_171
45845,case_999,31367,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,0.018454,0,8,normal,NaN,0,var_171


In [4]:
input = {
    "Case" : "caseid",
    "Activity" : "activity" ,
    "Resource" : "resource" ,
    "Timestamp": "ts",
    "resource_anomaly_type" : "y"  #Needs to be 1 for each Case
}

# Change columns name
df.rename(columns=input,inplace = True)

# Timestamp value in df
df["ts"] = pd.to_datetime(df["ts"])

In [5]:
# def dt1(df):
#     df['delta_time'] = df.groupby('index')['ts'].shift(-1) - df['ts']
#     df['delta_time'] = df['delta_time'].fillna(pd.Timedelta(seconds=0)) # ? how to fill nan
#     return df['delta_time'].tolist()

# df["dt"] = dt1(df) 
# #df.drop("ts",axis=1,inplace = True)

# df.head()

In [6]:
# drop ID-columns:

drop_id_threshold = 0.5

df = Utils.drop_id_columns(df, drop_id_threshold)

df

,caseid,activity,ts,resource,Resource_failure_rate,Resource_Pass/Fail,order,y,resource_parameter,trace_change_resource,variant_num
0,case_0,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358
...,...,...,...,...,...,...,...,...,...,...,...
45842,case_999,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_171
45843,case_999,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,0.007068,0,6,normal,NaN,0,var_171
45844,case_999,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,0.018454,0,7,normal,NaN,0,var_171
45845,case_999,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,0.018454,0,8,normal,NaN,0,var_171


## Data preprocessing

In [7]:
df["dt"] = Utils.dt(df) # This needs to be improved
df.drop("ts", axis=1,inplace = True)

In [8]:
Utils.persantage_nan(df)

caseid                    0.000000
activity                  0.000000
resource                  0.000000
Resource_failure_rate     0.000000
Resource_Pass/Fail        0.000000
order                     0.000000
y                         0.000000
resource_parameter       83.957511
trace_change_resource     0.000000
variant_num               0.000000
dt                        0.000000
dtype: float64

In [9]:
df = Utils.predict_null_value(colum="resource_parameter",df=df)

In [10]:
df = Utils.prod_nan_with_treshold(df)
df.head()

,caseid,activity,resource,Resource_failure_rate,Resource_Pass/Fail,order,y,resource_parameter,trace_change_resource,variant_num,dt,resource_parameter_was_null
0,case_0,Activity A,Resource_Group2_res_1,0.026984,0,1,normal,"loc = 5, len = 1.0",0,var_358,0 days 01:00:00,True
1,case_0,Activity B,Resource_Group0_res_0,0.018454,0,2,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True
2,case_0,Activity C,Resource_Group1_res_0,0.002551,0,3,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True
3,case_0,Activity D,Resource_Group2_res_2,0.029775,0,4,normal,"loc = 5, len = 2.0",0,var_358,0 days 01:00:00,True
4,case_0,Activity E,Resource_Group0_res_0,0.018454,0,5,normal,"loc = 7, len = 3.0",0,var_358,0 days 01:00:00,True


In [11]:
df_grouped = df.groupby(["caseid"]).agg(list).reset_index()

In [12]:
df_grouped = Utils.reduce_list_columns(df_grouped)

In [13]:
Utils.display_columns_to_aggragate(df_grouped)

,activity,resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_parameter,dt
0,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7]","[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
1,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[loc = 3, len = 2.0, loc = 3, len = 2.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 00:0..."
2,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7]","[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
3,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
...,...,...,...,...,...,...,...
4995,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4996,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_2, Resource_Group0_res_0,...","[0.029775339741158, 0.0184536333969282, 0.0070...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 2, len = 2.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4997,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."
4998,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[loc = 7, len = 3.0, loc = 7, len = 3.0, loc =...","[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0..."


In [14]:
#Some colomns have no importance in the process so we drop them
df_grouped = df_grouped.drop(['order'],axis=1) # domain knowledge
# df_grouped = df_grouped.drop(['Resource_failure_rate','Resource_Pass/Fail'],axis=1)

In [15]:
df_grouped.head(2)

,caseid,activity,resource,Resource_failure_rate,Resource_Pass/Fail,y,resource_parameter,trace_change_resource,variant_num,dt,resource_parameter_was_null
0,case_0,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_1, Resource_Group0_res_0,...","[0.0269841320197102, 0.0184536333969282, 0.002...","[0, 0, 0, 0, 0, 0, 0]",normal,"[loc = 5, len = 1.0, loc = 7, len = 3.0, loc =...",0,var_358,"[0 days 01:00:00, 0 days 01:00:00, 0 days 01:0...",True
1,case_1,"[Activity A, Activity B, Activity C, Activity ...","[Resource_Group2_res_0, Resource_Group0_res_0,...","[0.0035008269981937, 0.0184536333969282, 0.007...","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]",insert,"[loc = 3, len = 2.0, loc = 3, len = 2.0, loc =...",1,var_406,"[0 days 01:00:00, 0 days 01:00:00, 0 days 00:0...",False


In [16]:
# from statistics import mode, mean

# df_grouped_t = df_grouped.copy()

# for col in df_grouped.columns:    
#     #the columns need to have only list elements
#     x = df_grouped[col].iloc[0]

#     if (col != "dt"):
#         if(type(x) == list):
#             #if there is a float inside the list we should make avg,max,min
#             if type(x[0]) == float:
#                 df_grouped_t[col + "_avg"] = df_grouped_t[col].map(lambda x : mean(x))
#                 df_grouped_t[col + "_max"] = df_grouped_t[col].map(lambda x : max(x))
#                 df_grouped_t[col + "_min"] = df_grouped_t[col].map(lambda x : min(x))
#                 #df_grouped_t.drop(col,axis=1,inplace = True)

#             #if there is an int we do mode inside the list we should make mode
#             if type(x[0]) == int:
#                 df_grouped_t[col + "_mode"] = df_grouped_t[col].map(lambda x : mode(x))
#                 #df_grouped_t.drop(col,axis=1,inplace = True)
            
#             if type(col) not in [float, int, pd._libs.tslibs.timedeltas.Timedelta]:
#                 for el in df[col].unique():
#                     df_grouped_t[el] = df_grouped_t[col].map(lambda x : x.count(el)/len(x)) # frequency of an element
#                 df_grouped_t.drop(col,axis=1, inplace = True)

# df_grouped_t["avg_dt"] = df_grouped_t["dt"].map(lambda x : np.mean(x))
# df_grouped_t["max_dt"] = df_grouped_t["dt"].map(lambda x : np.max(x))
# df_grouped_t["min_dt"] = df_grouped_t["dt"].map(lambda x : np.min(x))

# df_grouped_t.drop("dt", axis=1, inplace = True)

# df_grouped_t

In [17]:
df_grouped_agg = Utils.aggregation_encoding(df_grouped,df)
df_grouped_agg

,caseid,y,trace_change_resource,variant_num,resource_parameter_was_null,Activity A,Activity B,Activity C,Activity D,Activity E,...,"loc = 3, len = 1.0","loc = 2, len = 3.0","loc = 8, len = 3.0","loc = 6, len = 1.0","loc = 8, len = 2.0","loc = 6, len = 3.0","loc = 8, len = 1.0",avg_dt,max_dt,min_dt
0,case_0,normal,0,var_358,True,0.142857,0.142857,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
1,case_1,insert,1,var_406,False,0.083333,0.083333,0.083333,0.083333,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 00:50:00,0 days 01:00:00,0 days 00:07:35
2,case_10,normal,0,var_358,True,0.142857,0.142857,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
3,case_100,normal,0,var_273,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
4,case_1000,normal,0,var_327,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,case_995,normal,0,var_582,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
4996,case_996,normal,0,var_529,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
4997,case_997,normal,0,var_582,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00
4998,case_998,normal,0,var_529,True,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0 days 01:00:00,0 days 01:00:00,0 days 01:00:00


In [21]:
# what to do with "variant_num": either label encoder or drop the column

## Logs encoding

## Model selection

## Evaluation of results